# Structured Outputs using Groq

This notebook demonstrates how to use Groq for inference instead of OpenAI or Anthropic. The examples use the mixtral-8x7b model.

## GroqCloud API

To use Groq, you need to obtain a Groq API key. Go to GroqCloud and login. Select API Keys from the left menu and then select Create API key to create a new key.

## Setup

First, let's install the required packages and set up the environment:

In [1]:
# %pip install instructor groq pydantic openai anthropic -qq

In [2]:
from pydantic import BaseModel, Field
from typing import List
import instructor

In [3]:
# API KEYS --> Make sure to include GROQ_API_KEY in api_keys.env file
from dotenv import load_dotenv
load_dotenv('../api_keys.env')

True

In [4]:
from groq import Groq
client = Groq()
client = instructor.from_groq(client, mode=instructor.Mode.TOOLS)

## Example Usage

In [5]:
class Character(BaseModel):
    name: str
    fact: List[str] = Field(..., description="A list of facts about the subject")


resp = client.chat.completions.create(
    model="mixtral-8x7b-32768",
    messages=[
        {
            "role": "user",
            "content": "Tell me about the company Tesla",
        }
    ],
    response_model=Character,
)

print(resp.model_dump_json(indent=2))

{
  "name": "Tesla",
  "fact": [
    "American electric vehicle and clean energy company",
    "Founded in 2003 by Elon Musk",
    "Headquartered in Austin, Texas",
    "Known for producing the Tesla Model S, Model X, Model 3, and Model Y"
  ]
}


Another example:

In [14]:
class UserExtract(BaseModel):
    name: str
    age: int


user: UserExtract = client.chat.completions.create(
    model="mixtral-8x7b-32768",
    response_model=UserExtract,
    messages=[
        {"role": "user", "content": "Paul is 64 years old"},
    ],
)

In [15]:
assert isinstance(user, UserExtract), "Should be instance of UserExtract"
assert user.name.lower() == "paul"
assert user.age == 64

print(user.model_dump_json(indent=2))

{
  "name": "Paul",
  "age": 64
}
